In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from scipy import stats
import time
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

class RandomForestWithUncertainty:
    """
    Random Forest regressor that provides uncertainty estimates
    for resistivity prediction in hydrologic surveys
    """
    
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, 
                 min_samples_leaf=1, random_state=42):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state
        self.rf = None
        self.is_fitted = False
        self.feature_scaler = None
        self.output_scaler = None
        
    def fit(self, X, y):
        """
        Fit Random Forest model with scalers
        """
        self.rf = RandomForestRegressor(
            n_estimators=self.n_estimators,
            max_depth=self.max_depth,
            min_samples_split=self.min_samples_split,
            min_samples_leaf=self.min_samples_leaf,
            random_state=self.random_state,
            oob_score=True,  # Enable out-of-bag scoring
            n_jobs=-1
        )
        
        self.rf.fit(X, y)
        self.is_fitted = True
        
        # Store training data for nearest neighbor uncertainty
        self.X_train = X.copy()
        self.y_train = y.copy()
        
        return self
    
    def predict_with_uncertainty(self, X, method='ensemble_variance', 
                                confidence_level=0.95):
        """
        Predict resistivity with uncertainty estimates
        
        Returns:
        --------
        predictions : array, shape (n_samples,)
            Mean resistivity predictions
        uncertainties : array, shape (n_samples,)
            Uncertainty estimates (standard deviation)
        prediction_intervals : array, shape (n_samples, 2)
            Lower and upper bounds of prediction intervals
        """
        
        if not self.is_fitted:
            raise ValueError("Model must be fitted before making predictions")
        
        if method == 'ensemble_variance':
            return self._ensemble_variance_uncertainty(X, confidence_level)
        elif method == 'nearest_neighbor':
            return self._nearest_neighbor_uncertainty(X, confidence_level)
        elif method == 'combined':
            return self._combined_uncertainty(X, confidence_level)
        else:
            raise ValueError(f"Unknown uncertainty method: {method}")
    
    def _ensemble_variance_uncertainty(self, X, confidence_level):
        """
        Calculate uncertainty from ensemble variance
        """
        # Get predictions from all trees
        tree_predictions = np.array([
            tree.predict(X) for tree in self.rf.estimators_
        ])
        
        # Calculate statistics
        predictions = np.mean(tree_predictions, axis=0)
        uncertainties = np.std(tree_predictions, axis=0)
        
        # Calculate prediction intervals
        z_score = stats.norm.ppf((1 + confidence_level) / 2)
        margin = z_score * uncertainties
        
        prediction_intervals = np.column_stack([
            predictions - margin,
            predictions + margin
        ])
        
        return predictions, uncertainties, prediction_intervals
    
    def _nearest_neighbor_uncertainty(self, X, confidence_level, k=10):
        """
        Calculate uncertainty based on local neighborhood variance
        """
        # Fit nearest neighbors on training data
        nn = NearestNeighbors(n_neighbors=min(k, len(self.X_train)))
        nn.fit(self.X_train)
        
        # Find nearest neighbors for each prediction point
        distances, indices = nn.kneighbors(X)
        
        # Get base predictions
        predictions = self.rf.predict(X)
        uncertainties = np.zeros(len(X))
        
        # Calculate local uncertainty for each point
        for i in range(len(X)):
            # Get predictions for nearest neighbors
            neighbor_indices = indices[i]
            if hasattr(self.y_train, 'iloc'):
                neighbor_targets = self.y_train.iloc[neighbor_indices]
            else:
                neighbor_targets = self.y_train[neighbor_indices]
            
            # Use variance of neighbors + distance penalty as uncertainty estimate
            local_variance = np.var(neighbor_targets)
            distance_penalty = np.mean(distances[i]) * 0.1  # Scale distance penalty
            uncertainties[i] = np.sqrt(local_variance + distance_penalty)
        
        # Calculate prediction intervals
        z_score = stats.norm.ppf((1 + confidence_level) / 2)
        margin = z_score * uncertainties
        
        prediction_intervals = np.column_stack([
            predictions - margin,
            predictions + margin
        ])
        
        return predictions, uncertainties, prediction_intervals
    
    def _combined_uncertainty(self, X, confidence_level):
        """
        Combine ensemble variance and nearest neighbor uncertainty
        """
        # Get both uncertainty estimates
        pred1, unc1, _ = self._ensemble_variance_uncertainty(X, confidence_level)
        pred2, unc2, _ = self._nearest_neighbor_uncertainty(X, confidence_level)
        
        # Use ensemble prediction (more stable)
        predictions = pred1
        
        # Combine uncertainties (weighted average)
        uncertainties = np.sqrt(0.7 * unc1**2 + 0.3 * unc2**2)
        
        # Calculate prediction intervals
        z_score = stats.norm.ppf((1 + confidence_level) / 2)
        margin = z_score * uncertainties
        
        prediction_intervals = np.column_stack([
            predictions - margin,
            predictions + margin
        ])
        
        return predictions, uncertainties, prediction_intervals
    
    def identify_high_uncertainty_locations(self, X, uncertainty_threshold_percentile=80):
        """
        Identify locations with high uncertainty for targeted sampling
        """
        predictions, uncertainties, _ = self.predict_with_uncertainty(X, method='combined')
        
        # Define high uncertainty threshold
        threshold = np.percentile(uncertainties, uncertainty_threshold_percentile)
        high_uncertainty_mask = uncertainties >= threshold
        
        # Rank locations by uncertainty
        uncertainty_ranking = np.argsort(uncertainties)[::-1]  # Highest first
        
        return high_uncertainty_mask, uncertainty_ranking, uncertainties

# Load the terrain attributes CSV file
FILE_PATH = "terrain_attributes_clean.csv"
print(f"Loading terrain data from {FILE_PATH}...")
df = pd.read_csv(FILE_PATH)

# Explore the dataset
print("\nDataset Info:")
print(f"Shape: {df.shape}")

# Identify depth layers (columns starting with 'layer_')
depth_layers = [col for col in df.columns if col.startswith('layer_')]
print(f"\nFound {len(depth_layers)} depth layers: {depth_layers}")

# Feature columns (terrain attributes)
feature_columns = ['elevation', 'slope', 'aspect', 'plan_curvature']
print(f"\nFeature columns: {feature_columns}")

# Function to safely apply log10 transformation to resistivity data
def safe_log10_transform(data):
    """Safely apply log10 transformation to data, handling zeros and negative values."""
    # Make a copy to avoid modifying the original
    if isinstance(data, pd.Series):
        data_copy = data.copy()
    else:
        data_copy = np.copy(data)
    
    # Check for non-positive values
    min_val = np.nanmin(data_copy)
    offset = 0
    
    # If we have zero or negative values, add an offset
    if min_val <= 0:
        offset = abs(min_val) + 1e-10  # Add a small epsilon
        data_copy = data_copy + offset
        print(f"Added offset of {offset} before log transform (min value was {min_val})")
    
    # Apply log10 transform and handle infinities in one step
    with np.errstate(divide='ignore', invalid='ignore'):
        log_data = np.log10(data_copy)
        # Replace infinities with NaN
        if isinstance(log_data, np.ndarray):
            log_data[np.isinf(log_data)] = np.nan
        elif isinstance(log_data, pd.Series):
            log_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    return log_data, offset

# Function to inverse the log10 transformation
def inverse_log10_transform(log_data, offset):
    """Inverse the log10 transformation, removing any offset added."""
    # Handle infinities
    if isinstance(log_data, np.ndarray):
        clean_log_data = log_data.copy()
        clean_log_data[np.isinf(clean_log_data)] = np.nan
    elif isinstance(log_data, pd.Series):
        clean_log_data = log_data.replace([np.inf, -np.inf], np.nan)
    else:
        clean_log_data = log_data
        
    # Apply inverse transform
    return 10**clean_log_data - offset

# Function to visualize feature importance for a layer
def plot_feature_importance(model, feature_columns, layer_name):
    """Create a feature importance plot for the trained model."""
    # Get feature importances
    importances = model.rf.feature_importances_  # Updated for uncertainty model
    indices = np.argsort(importances)[::-1]
    
    # Create a figure for the feature importance
    plt.figure(figsize=(10, 6))
    plt.title(f'Feature Importances for {layer_name}')
    plt.bar(range(len(feature_columns)), importances[indices], align='center')
    plt.xticks(range(len(feature_columns)), [feature_columns[i] for i in indices], rotation=45)
    plt.tight_layout()
    
    # Save the figure
    plt.savefig(f"{layer_name}_feature_importance.png", dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved feature importance plot to {layer_name}_feature_importance.png")

# Function to visualize spatial distribution of resistivity with uncertainty
def plot_spatial_distribution_with_uncertainty(df, layer_name):
    """Create spatial distribution plots for predictions and uncertainty."""
    uncertainty_col = f"{layer_name}_uncertainty"
    
    # Skip if the layer has too many NaN values
    if df[layer_name].isna().sum() > 0.9 * len(df):
        print(f"Too many NaN values to create spatial distribution plot for {layer_name}")
        return
    
    # Make a copy to avoid modifying original data
    plot_df = df.copy()
    
    # Replace any inf values with NaN
    for col in [layer_name, uncertainty_col]:
        if col in plot_df.columns:
            if isinstance(plot_df[col], pd.Series):
                plot_df[col] = plot_df[col].replace([np.inf, -np.inf], np.nan)
            else:
                plot_df.loc[np.isinf(plot_df[col]), col] = np.nan
    
    # Create subplot for predictions and uncertainty
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    # Plot 1: Resistivity predictions
    valid_mask1 = ~plot_df[layer_name].isna()
    if valid_mask1.sum() > 0:
        sc1 = ax1.scatter(plot_df.loc[valid_mask1, 'x'], 
                         plot_df.loc[valid_mask1, 'y'], 
                         c=plot_df.loc[valid_mask1, layer_name], 
                         cmap='viridis', alpha=0.7, s=20)
        plt.colorbar(sc1, ax=ax1, label=f'Resistivity at {layer_name}')
        ax1.set_xlabel('X Coordinate')
        ax1.set_ylabel('Y Coordinate')
        ax1.set_title(f'Resistivity Predictions - {layer_name}')
    
    # Plot 2: Uncertainty
    if uncertainty_col in plot_df.columns:
        valid_mask2 = ~plot_df[uncertainty_col].isna()
        if valid_mask2.sum() > 0:
            sc2 = ax2.scatter(plot_df.loc[valid_mask2, 'x'], 
                             plot_df.loc[valid_mask2, 'y'], 
                             c=plot_df.loc[valid_mask2, uncertainty_col], 
                             cmap='Reds', alpha=0.7, s=20)
            plt.colorbar(sc2, ax=ax2, label=f'Prediction Uncertainty')
            ax2.set_xlabel('X Coordinate')
            ax2.set_ylabel('Y Coordinate')
            ax2.set_title(f'Prediction Uncertainty - {layer_name}')
            
            # Identify high uncertainty areas
            threshold = plot_df[uncertainty_col].quantile(0.8)
            high_unc_mask = plot_df[uncertainty_col] >= threshold
            if high_unc_mask.sum() > 0:
                ax2.scatter(plot_df.loc[high_unc_mask, 'x'], 
                           plot_df.loc[high_unc_mask, 'y'], 
                           c='red', marker='x', s=100, alpha=0.8,
                           label=f'Top 20% Uncertainty (n={high_unc_mask.sum()})')
                ax2.legend()
    
    plt.tight_layout()
    
    # Save the figure
    plt.savefig(f"{layer_name}_spatial_distribution_with_uncertainty.png", 
                dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved spatial distribution plot to {layer_name}_spatial_distribution_with_uncertainty.png")

# Function to save optimal survey locations
def save_optimal_survey_locations(df, layer_name, n_locations=10):
    """Identify and save optimal locations for future surveys based on uncertainty."""
    uncertainty_col = f"{layer_name}_uncertainty"
    
    if uncertainty_col not in df.columns:
        print(f"No uncertainty data available for {layer_name}")
        return
    
    # Get valid uncertainty data
    valid_data = df.dropna(subset=[uncertainty_col, 'x', 'y'])
    
    if len(valid_data) < n_locations:
        print(f"Not enough valid uncertainty data for {layer_name}")
        return
    
    # Sort by uncertainty (highest first)
    optimal_locations = valid_data.nlargest(n_locations, uncertainty_col)
    
    # Save to CSV
    optimal_locations[['x', 'y', uncertainty_col, layer_name]].to_csv(
        f"{layer_name}_optimal_survey_locations.csv", index=False
    )
    
    print(f"Saved top {len(optimal_locations)} optimal survey locations for {layer_name}")
    print(f"Mean uncertainty at optimal locations: {optimal_locations[uncertainty_col].mean():.4f}")
    print(f"Mean uncertainty overall: {df[uncertainty_col].mean():.4f}")

# Function to train model and predict for a specific depth layer with uncertainty
def train_and_predict_for_layer_with_uncertainty(df, layer_name, feature_columns):
    start_time = time.time()
    print(f"\n--- Processing {layer_name} with Uncertainty Estimation ---")
    
    # Split data into known and unknown sets
    known_data = df.dropna(subset=[layer_name]).copy()
    unknown_data = df[df[layer_name].isna()].copy()
    
    print(f"Training data size: {known_data.shape[0]} rows")
    print(f"Prediction data size: {unknown_data.shape[0]} rows")
    
    if known_data.shape[0] < 10:
        print(f"WARNING: Not enough training data for {layer_name}. Skipping.")
        return df
    
    # Further filter known data to only include rows where all features are non-NaN
    known_data = known_data.dropna(subset=feature_columns)
    print(f"Training data size after removing NaN features: {known_data.shape[0]} rows")
    
    if known_data.shape[0] < 10:
        print(f"WARNING: Not enough non-NaN training data for {layer_name}. Skipping.")
        return df
    
    # Extract features and target
    X = known_data[feature_columns]
    y = known_data[layer_name]
    
    # First apply log10 transformation to the target variable
    print(f"Applying log10 transformation to {layer_name}")
    y_log, log_offset = safe_log10_transform(y)
    
    # Remove NaN values before scaling
    valid_mask = ~np.isnan(y_log)
    if not np.any(valid_mask):
        print(f"WARNING: All values became NaN after log transform for {layer_name}. Skipping.")
        return df
    
    # Extract valid data points
    X_valid = X[valid_mask]
    y_log_valid = y_log[valid_mask]
    
    # EXPLICITLY normalize input features to 0-1 range
    feature_scaler = MinMaxScaler(feature_range=(0, 1))
    X_normalized = feature_scaler.fit_transform(X_valid)
    
    # Print before and after normalization stats
    print("\nInput features before normalization:")
    for i, col in enumerate(feature_columns):
        col_values = X_valid[col].values
        print(f"  {col}: min={np.min(col_values):.4f}, max={np.max(col_values):.4f}, mean={np.mean(col_values):.4f}")
    
    print("\nInput features after normalization:")
    for i, col in enumerate(feature_columns):
        col_values = X_normalized[:, i]
        print(f"  {col}: min={np.min(col_values):.4f}, max={np.max(col_values):.4f}, mean={np.mean(col_values):.4f}")
    
    # Create output scaler for the log-transformed target variable (0-1 range)
    output_scaler = MinMaxScaler(feature_range=(0, 1))
    
    # Properly reshape to 2D array for scaling
    y_log_valid_2d = y_log_valid.values.reshape(-1, 1)
    y_normalized_valid = output_scaler.fit_transform(y_log_valid_2d).flatten()
    
    # Split the data with valid values only
    X_train, X_test, y_train_norm, y_test_norm = train_test_split(
        X_normalized, y_normalized_valid, test_size=0.2, random_state=42
    )
    
    # Also keep track of original values for evaluation
    _, _, y_train_orig, y_test_orig = train_test_split(
        X_valid, y[valid_mask], test_size=0.2, random_state=42
    )
    
    # Train the RandomForest model with uncertainty estimation
    print("\nTraining model with uncertainty estimation...")
    model = RandomForestWithUncertainty(
        n_estimators=100, 
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42
    )
    model.fit(X_train, y_train_norm)
    
    # Store scalers in model for later use
    model.feature_scaler = feature_scaler
    model.output_scaler = output_scaler
    model.log_offset = log_offset
    
    # Generate feature importance plot
    plot_feature_importance(model, feature_columns, layer_name)
    
    # Evaluate on test set with uncertainty
    y_pred_norm, y_uncertainty_norm, y_intervals_norm = model.predict_with_uncertainty(
        X_test, method='combined'
    )
    
    # Convert normalized predictions back to original scale
    # First to log scale
    y_pred_log = output_scaler.inverse_transform(y_pred_norm.reshape(-1, 1)).flatten()
    y_uncertainty_log = y_uncertainty_norm * (output_scaler.data_max_ - output_scaler.data_min_)
    
    # Then to original scale
    y_pred_orig = inverse_log10_transform(y_pred_log, log_offset)
    
    # Calculate metrics using original scale
    mse = mean_squared_error(y_test_orig, y_pred_orig)
    r2 = r2_score(y_test_orig, y_pred_orig)
    mean_uncertainty = np.mean(y_uncertainty_norm)

    
    print(f"Model evaluation (original scale):")
    print(f"Mean Squared Error: {np.sqrt(mse):.4f}")
    print(f"R² Score: {r2:.4f}")
    print(f"Mean Prediction Uncertainty (normalized): {mean_uncertainty:.4f}")
    
    # Calculate uncertainty calibration (what % of true values fall within prediction intervals)
    y_intervals_log = output_scaler.inverse_transform(y_intervals_norm.reshape(-1, 2))
    y_intervals_orig = np.column_stack([
        inverse_log10_transform(y_intervals_log[:, 0], log_offset),
        inverse_log10_transform(y_intervals_log[:, 1], log_offset)
    ])
    
    coverage = np.mean((y_test_orig >= y_intervals_orig[:, 0]) & 
                      (y_test_orig <= y_intervals_orig[:, 1]))
    print(f"Prediction Interval Coverage: {coverage:.2%} (target: 95%)")
    
    # Predict unknown values with uncertainty
    if unknown_data.shape[0] > 0:
        # Create a mask for rows with all non-NaN features
        valid_features_mask = unknown_data[feature_columns].notna().all(axis=1)
        print(f"Found {valid_features_mask.sum()} out of {unknown_data.shape[0]} unknown rows with all features non-NaN")
        
        # Only make predictions for rows with all non-NaN features
        if valid_features_mask.sum() > 0:
            # Extract data for prediction
            valid_unknown_data = unknown_data[valid_features_mask]
            X_unknown = valid_unknown_data[feature_columns]
            
            # Normalize the unknown features using the same scaler
            X_unknown_normalized = feature_scaler.transform(X_unknown)
            
            # Make predictions with uncertainty (in normalized space)
            unknown_pred_norm, unknown_uncertainty_norm, unknown_intervals_norm = model.predict_with_uncertainty(
                X_unknown_normalized, method='combined'
            )
            
            # Convert predictions back to original scale
            # First to log scale
            unknown_pred_log = output_scaler.inverse_transform(
                unknown_pred_norm.reshape(-1, 1)
            ).flatten()
            
            # Then to original scale
            unknown_predictions = inverse_log10_transform(unknown_pred_log, log_offset)
            
            # Scale uncertainty back to log scale
            unknown_uncertainty_log = unknown_uncertainty_norm * (output_scaler.data_max_ - output_scaler.data_min_)
            
            # Create a copy of the original dataframe
            updated_df = df.copy()
            
            # Update only the NaN values with predictions for rows where all features are non-NaN
            mask = updated_df[layer_name].isna() & updated_df[feature_columns].notna().all(axis=1)
            updated_df.loc[mask, layer_name] = unknown_predictions
            updated_df.loc[mask, f"{layer_name}_uncertainty"] = unknown_uncertainty_log
            
            print(f"Filled {sum(mask)} missing values for {layer_name}")
            print(f"Left {sum(updated_df[layer_name].isna())} values as NaN (either target or features had NaN)")
            
            # Generate the spatial distribution plot with uncertainty
            plot_spatial_distribution_with_uncertainty(updated_df, layer_name)
            
            # Save optimal survey locations
            save_optimal_survey_locations(updated_df, layer_name, n_locations=10)
            
            print(f"Processing time: {time.time() - start_time:.2f} seconds")
            return updated_df
        else:
            print(f"No valid rows with all non-NaN features to predict for {layer_name}")
            return df
    else:
        print(f"No missing values to predict for {layer_name}")
        return df

# Process each depth layer with uncertainty estimation
updated_df = df.copy()

for layer in depth_layers:
    updated_df = train_and_predict_for_layer_with_uncertainty(updated_df, layer, feature_columns)

# Save the updated dataset with predictions and uncertainties
output_file = "terrain_with_predicted_resistivity_and_uncertainty.csv"
updated_df.to_csv(output_file, index=False)
print(f"\nSaved complete dataset with predictions and uncertainties to {output_file}")

# Print summary of filled values and uncertainty statistics
print("\nSummary of filled values and uncertainty:")
for layer in depth_layers:
    original_nan_count = df[layer].isna().sum()
    remaining_nan_count = updated_df[layer].isna().sum()
    filled_count = original_nan_count - remaining_nan_count
    total_count = len(df)
    
    uncertainty_col = f"{layer}_uncertainty"
    if uncertainty_col in updated_df.columns:
        mean_uncertainty = updated_df[uncertainty_col].mean()
        max_uncertainty = updated_df[uncertainty_col].max()
        print(f"{layer}: Filled {filled_count}/{original_nan_count} missing values ({filled_count/total_count*100:.1f}% of total)")
        print(f"  Mean uncertainty: {mean_uncertainty:.4f}, Max uncertainty: {max_uncertainty:.4f}")
    else:
        print(f"{layer}: Filled {filled_count}/{original_nan_count} missing values ({filled_count/total_count*100:.1f}% of total)")

# Create summary report for ModEx framework
print("\n" + "="*60)
print("MODEX FRAMEWORK SUMMARY REPORT")
print("="*60)

print(f"\nDataset Overview:")
print(f"Total locations: {len(updated_df)}")
print(f"Depth layers processed: {len(depth_layers)}")
print(f"Features used: {feature_columns}")

for layer in depth_layers:
    uncertainty_col = f"{layer}_uncertainty"
    if uncertainty_col in updated_df.columns:
        print(f"\n{layer.upper()} - ModEx Recommendations:")
        
        # High uncertainty locations for targeted surveys
        high_unc_threshold = updated_df[uncertainty_col].quantile(0.8)
        high_unc_count = (updated_df[uncertainty_col] >= high_unc_threshold).sum()
        
        print(f"  High uncertainty locations (top 20%): {high_unc_count}")
        print(f"  Recommended for Phase 5 field campaigns")
        print(f"  Priority: {'HIGH' if high_unc_count > 50 else 'MEDIUM' if high_unc_count > 20 else 'LOW'}")
        
        # Coverage assessment
        predicted_count = updated_df[layer].notna().sum()
        coverage_pct = predicted_count / len(updated_df) * 100
        print(f"  Spatial coverage: {coverage_pct:.1f}%")
        print(f"  Model confidence: {'HIGH' if coverage_pct > 80 else 'MEDIUM' if coverage_pct > 60 else 'LOW'}")

print(f"\nNext Steps for ModEx Cycle:")
print(f"1. Review optimal survey locations CSV files")
print(f"2. Plan Phase 5 field campaigns for high uncertainty areas") 
print(f"3. Collect new resistivity data at recommended locations")
print(f"4. Retrain models (Phase 6) with new data to reduce uncertainty")
print(f"5. Iterate until acceptable uncertainty levels achieved")

print("\nDone!")